# KIM models

This Notebook provides more details on how the potentials package (starting with version 0.3.0) handles and interacts with OpenKIM models.

In [1]:
# https://github.com/lmhale99/potentials
import potentials

print('Notebook tested for potentials version', potentials.__version__)

Notebook tested for potentials version 0.4.0


## 1. Overview of OpenKIM models in potentials

The OpenKIM project manages interatomic potentials differently than the native LAMMPS potentials that are directly archived in the NIST repository.
- While LAMMPS reads in potential parameters or parameter files associated with a given functional form, OpenKIM "models" exist as pre-compiled code.
- Using OpenKIM models therefore requires that the kim-api be installed, LAMMPS built to interact with it, and for the various OpenKIM models you wish to use to be downloaded and installed through the kim-api.  See the https://openkim.org/ website for more details.
- OpenKIM uses its own identification and version conventions to name the various hosted models.  The available model versions can depend on the kim-api version being used, which in turn can depend on the version of LAMMPS being used. 

The potentials package provides a means of interacting with the OpenKIM models in the same fashion as native LAMMPS potentials.  This is managed using the following three things

1. The potential_LAMMPS_KIM records collect metadata for OpenKIM models that connect the model to one or more potential listings in the NIST database.  Each record is named after the OpenKIM model's shortcode, which uniquely defines the model without version information.
2. A PotentialLAMMPSKIM object can then be generated from a potential_LAMMPS_KIM record by supplying the specific OpenKIM full id (info+shortcode+version). These objects provide methods and attributes for the OpenKIM model that are comparable to those provided by PotentialLAMMPS objects for native LAMMPS potentials.
3. When calling get_lammps_potential(s), the included list of OpenKIM models is built based on either a pre-defined list of full OpenKIM ids or by using the kim-api to retrieve the list of installed models.


## 2. List of installed OpenKIM models

The Database class manages the list of OpenKIM models to build records for when get_lammps_potentials() is called.  This list can be viewed using the __Database.kim_models__ attribute.

In [2]:
potdb = potentials.Database()
print(len(potdb.kim_models), 'kim models')
for i, kim_model in enumerate(potdb.kim_models):
    if i == 5:
        print('...')
        break
    print(kim_model)
    

1 kim models
EAM_CubicNaturalSpline_AngeloMoody_1995_Ni__MO_800536961967_002


### 2.1. Default OpenKIM model lists

The saved settings allow for either a kim_api_directory value or a list of kim_models to be saved and used to generate the default list of kim_models. The associated settings commands are:

- __settings.kim_api_directory__ is the path to the root directory of the installed KIM API version to use as the default.  If set, then the kim_models list will be generated by calling the API and retrieving the list of currently installed OpenKIM models.
- __settings.set_kim_api_directory(path=None)__ adds or updates the default kim_api_directory value.  If path is not given, then a screen prompt will ask for one.
- __settings.unset_kim_api_directory()__ removes the previously saved default kim_api_directory value.
- __settings.kim_models_file__ returns the location of the default kim_models_file.  The OpenKIM models listed in this file will be read in if settings.kim_api_directory is not set.  Note that this file location cannot be changed.
- __settings.set_kim_models(kim_models, kim_models_file=None )__ saves the given kim_models list to kim_models_file.  If kim_models_file is not specified, then the list of models is saved to settings.kim_models_file.
- __settings.unset_kim_models()__ deletes the list of kim_models stored in settings.kim_models_file.



In [3]:
print(potentials.settings.kim_api_directory)

/home/lmh1/LAMMPS/2022-06-23/lib/kim/installed-2.2.1/bin


In [4]:
print(potentials.settings.kim_models_file)

/home/lmh1/.NISTpotentials/kim_models.txt


### 2.2. Specifying OpenKIM models when initializing a Database

When initializing a Database object or calling get_lammps_potential(s), one of the following parameters can be given to specify the list of OpenKIM models to include:

- __kim_api_directory__ (*path-like object, optional*) is the root directory of the installed KIM API version that you plan on using for the simulations.  If given, the associated kim-api-collections-management will be called to determine the list of installed KIM models.
- __installed_kim_models__ (*list, optional*) is a list of full KIM ids.
- __kim_settings_file__ (*path-like object, optional*) is the name of a JSON/XML file that contains subelement 'installed-kim-models' that lists full KIM ids.

In [5]:
potdb = potentials.Database(kim_models=['EAM_Dynamo_AcklandTichyVitek_1987_Ag__MO_212700056563_005'])
potdb.kim_models

['EAM_Dynamo_AcklandTichyVitek_1987_Ag__MO_212700056563_005']

Specifying the kim_api_directory during Database initilization

In [6]:
potdb = potentials.Database(kim_api_directory='/home/lmh1/LAMMPS/2022-06-23/lib/kim/installed-2.2.1/bin')
print(len(potdb.kim_models), 'kim models')
for i, kim_model in enumerate(potdb.kim_models):
    if i == 5:
        print('...')
        break
    print(kim_model)

454 kim models
DUNN_WenTadmor_2019v1_C__MO_584345505904_000
DUNN_WenTadmor_2019v2_C__MO_956135237832_000
DUNN_WenTadmor_2019v3_C__MO_714772088128_000
EAM_CubicNaturalSpline_AngeloMoodyBaskes_1995_Ni__MO_800536961967_003
EAM_CubicNaturalSpline_ErcolessiAdams_1994_Al__MO_800509458712_002
...


Specifying a kim_settings_file during Database initialization

In [7]:
kim_models_file_contents = """
EAM_CubicNaturalSpline_AngeloMoody_1995_Ni__MO_800536961967_002
EAM_CubicNaturalSpline_ErcolessiAdams_1994_Al__MO_800509458712_002
EAM_Dynamo_AcklandBaconCalder_1997_Fe__MO_142799717516_005
"""

with open('kim_models.txt', 'w') as f:
    f.write(kim_models_file_contents)
   
    
potdb = potentials.Database(kim_models_file='kim_models.txt')
potdb.kim_models

['EAM_CubicNaturalSpline_AngeloMoody_1995_Ni__MO_800536961967_002',
 'EAM_CubicNaturalSpline_ErcolessiAdams_1994_Al__MO_800509458712_002',
 'EAM_Dynamo_AcklandBaconCalder_1997_Fe__MO_142799717516_005']

### 2.3. Other database operations

The rest of the database methods associated with OpenKIM models are all utility functions that are used by or relate to the methods listed above.

- __get_kim_lammps_potentials(...)__ takes the same arguments as get_lammps_potentials() and returns PotentialLAMMPSKIM objects based on the kim_models list.  As this is called by get_lammps_potentials(), there's no reason to call this method unless you explicitly want to use only OpenKIM models.
- __init_kim_models(kim_models=None, kim_api_directory=None, kim_models_file=None)__ updates the kim_models list using either one of the given parameters (as described in section 2.2), or the default settings if no parameters are given.
- __find_kim_models(kim_api_directory=None)__ updates the kim_models list by using the KIM-API in the given kim_api_directory value.  This is what init_kim_models() calls when kim_api_directory is given.
- __set_kim_models(kim_models)__ directly updates the kim_models list to the given value.  This is what init_kim_models() calls when kim_models is given.
- __load_kim_models_file(kim_models_file=None)__ updates the kim_models list by reading in the values in kim_models_file. This is what init_kim_models() calls when kim_models_file is given.
- __save_kim_models_file(kim_models_file=None)__ Identical to settings.set_kim_models() with the Database object's current kim_models file being the list of models saved to the file.
- __delete_kim_models_file()__ Identical to settings.unset_kim_models().

In [8]:
# Reset kim_models back to the default
potdb.init_kim_models()

# Get the first kim potential
lmppot = potdb.get_kim_lammps_potentials(verbose=True)[0]

Found 450 matching potential_LAMMPS_KIM records in local library


100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:03<00:00, 144.26it/s]


Found 450 matching potential_LAMMPS_KIM records in remote library
 - 0 remote records are new
Built 1 lammps potentials for KIM models


## 3. PotentialLAMMPSKIM class

The PotentialLAMMPSKIM class is designed to work as close to the PotentialLAMMPS class as possible.  

The two primary differences that users should be aware of are

1. While each native LAMMPS potential is associated with exactly one potential entry, some OpenKIM models are associated with multiple potential entries. This is the case for potentials based on library parameter files that contain parameters for many individual potentials, and for so-called "universal" potentials that provide a simple general representation of all elements. To manage this, the PotentialLAMMPSKIM objects have additional attributes of

    - __symbolsets__ being a list of lists, with each sublist indicating the sets of elemental symbols associated with each potential entry.  While element combinations outside these sets can be used, it is not recommended as the cross interactions have likely not been fit nor tested. 
    - __potids__ being a list of the potential entry ids associated with the OpenKIM model.
    - __potkeys__ being a list of the potential entry keys associated with the OpenKIM model.

2. The LAMMPS command lines returned by PotentialLAMMPSKIM.pair_info() should only be used for preview purposes and not actually used for generating scripts.  This is because the kim commands require system information to be defined between kim init and kim model.  Instead, use the method variations that include structure information

    - __pair_data_info()__, which generates the kim commands and commands for reading a structure data file.
    - __pair_restart_info()__, which generates the kim commands and commands for reading a restart file.